## Step 3.1 – Delivery Delay Distribution

In [ ]:
SELECT 
    delivery_status,
    COUNT(*) AS num_deliveries,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM delivery_logs) * 100, 2) AS pct_deliveries,
    ROUND(AVG(DATEDIFF(actual_delivery_date, expected_delivery_date)), 2) AS avg_delay_days
FROM delivery_logs
GROUP BY delivery_status
ORDER BY num_deliveries DESC;

**Insight:** ~37% of shipments are delayed, averaging 2.08 days late.

## Step 3.2 – Average Lead Time per Supplier

In [ ]:
SELECT 
    s.supplier_name,
    s.country,
    ROUND(AVG(DATEDIFF(actual_delivery_date, order_date)), 2) AS avg_lead_time_days,
    COUNT(*) AS num_orders
FROM delivery_logs d
JOIN suppliers s ON d.supplier_id = s.supplier_id
WHERE actual_delivery_date IS NOT NULL
GROUP BY s.supplier_name, s.country
ORDER BY avg_lead_time_days DESC
LIMIT 10;

**Insight:** All top suppliers have ~38–39 day lead times, consistent for high-value semiconductor components.

## Step 3.3 – Stock-Out Frequency

In [ ]:
SELECT 
    c.category,
    COUNT(*) AS stockout_days
FROM inventory_levels i
JOIN components c ON i.component_id = c.component_id
WHERE i.closing_stock = 0
GROUP BY c.category
ORDER BY stockout_days DESC;

**Insight:** High-volume consumables like Carrier Tapes/Trays have the most stock-out days, creating recurring micro-delays.

## Step 3.4 – Overstock vs. Safety Stock

In [ ]:
SELECT 
    c.category,
    COUNT(*) AS overstock_days,
    ROUND(COUNT(*) / (SELECT COUNT(*) FROM inventory_levels) * 100, 2) AS pct_overstock_days
FROM inventory_levels i
JOIN components c 
    ON i.component_id = c.component_id
WHERE i.closing_stock > c.safety_stock_units
GROUP BY c.category
ORDER BY overstock_days DESC;

**Insight:** Overstock is rare (<1% for most categories); the bigger risk is frequent stock-outs.

## Step 3.5 – Forecast Accuracy (MAPE) by Category

In [ ]:
SELECT 
    c.category,
    ROUND(AVG(ABS(f.forecast_units - po.actual_units) / f.forecast_units) * 100, 2) AS avg_forecast_error_pct
FROM forecasts f
JOIN (
    SELECT 
        DATE_FORMAT(date, '%Y-%m') AS month,
        component_id,
        SUM(units_required) AS actual_units
    FROM production_orders
    GROUP BY month, component_id
) po 
  ON f.month = po.month AND f.component_id = po.component_id
JOIN components c ON f.component_id = c.component_id
GROUP BY c.category
ORDER BY avg_forecast_error_pct DESC;

**Insight:** Forecast errors range from 20–23% across categories, indicating consistent bias/volatility.